In [1]:
# Initial Imports
import pandas as pd
import numpy as np
from pathlib import Path
from dotenv import load_dotenv
import time
from joblib import dump,load # Save Models
from numpy import random
import os
from datetime import date, datetime, timedelta
import matplotlib.pyplot as plt

In [39]:
# Read CSV into dataframes
btcusd_6h_historical_csv = Path('C:/users/gdepa/gitlab/mantis-trading/crypto-trading-system/df_candles_kraken_btcusd_6h_append_9102.csv')
btcusd_6h_historical = pd.read_csv(btcusd_6h_historical_csv,index_col="time")
# make sure rows are every 6hrs, if there is no row-then make one and forward fill data (shrimpy doesn't print candle if there is no tick)

ethbtc_6h_historical_csv = Path('C:/users/gdepa/gitlab/mantis-trading/crypto-trading-system/df_prices_kraken_ethbtc_6h_historical_20190202_20200925.csv')
ethbtc_6h_historical = pd.read_csv(ethbtc_6h_historical_csv, index_col="time")

In [3]:
# make sure rows are every 6hrs, if there is no row-then make one and forward fill data (shrimpy doesn't print candle if there is no tick)
def calc_feature_dataframe(prices_df):
    ## cumulative returns as velocity
    ## Log returns as velocity
    ## partials?
    ## Lags?
    ## stock to flow
    ## Technical Indicators
    ## Social Indicators

    df_features = prices_df
    # Construct dependent variable
    df_features['returns'] = df_features['close'].pct_change()
    # Calculate cumulative returns
    df_features['cum_returns'] = (df_features['returns']+1).cumprod()
    # ----------------------- Price Dynamics --------------------------------
    # price dynamics as a one Dimensional particle problem in physics
    df_features['price_velocity_2'] = df_features['close'].pct_change(2)
    df_features['price_velocity_3'] = df_features['close'].pct_change(3)
    df_features['price_velocity_4'] = df_features['close'].pct_change(4)
    df_features['price_velocity_7'] = df_features['close'].pct_change(7)
    df_features['price_velocity_30'] = df_features['close'].pct_change(30)
    
    df_features['price_acceleration_1'] = df_features['returns'].pct_change(1)
    df_features['price_acceleration_2'] = df_features['price_velocity_2'].pct_change(2)
    df_features['price_acceleration_3'] = df_features['price_velocity_3'].pct_change(3)
    df_features['price_acceleration_4'] = df_features['price_velocity_4'].pct_change(4)
    df_features['price_acceleration_7'] = df_features['price_velocity_7'].pct_change(7)
    df_features['price_acceleration_30'] = df_features['price_velocity_30'].pct_change(30)

    df_features['rolling_mean_velocity_2'] = df_features['returns'].rolling(window=2).mean()
    df_features['rolling_mean_velocity_3'] = df_features['returns'].rolling(window=3).mean()
    df_features['rolling_mean_velocity_4'] = df_features['returns'].rolling(window=4).mean()
    df_features['rolling_mean_velocity_7'] = df_features['returns'].rolling(window=7).mean()
    df_features['rolling_mean_velocity_30'] = df_features['returns'].rolling(window=30).mean()
    
    df_features['rolling_mean_acceleration_2'] = df_features['price_acceleration_1'].rolling(window=2).mean()
    df_features['rolling_mean_acceleration_3'] = df_features['price_acceleration_1'].rolling(window=3).mean()
    df_features['rolling_mean_acceleration_4'] = df_features['price_acceleration_1'].rolling(window=4).mean()
    df_features['rolling_mean_acceleration_7'] = df_features['price_acceleration_1'].rolling(window=7).mean()
    df_features['rolling_mean_acceleration_30'] = df_features['price_acceleration_1'].rolling(window=30).mean()
    # To extend space to entire line the log price is mapped to position x(t) in the space by
    # x(t) = log(S(t))   where S(t) is the price of the instrument
    df_features['log_price'] = np.log(df_features['close'])
    df_features['log_returns'] = df_features['log_price'].diff() # Diff or percent change
    #df_features['log_return_pct']  = df_features['log_price'].pct()
    #df_features['cum_log_returns'] =(df_features['log_returns_pct'] + 1).cumprod()
    # Assumption: Returns of financial instruments are lognormally distributed
    # v(t) = R(t) = dx(t)/dt where v(t) is the velocity of the instrument in the log price space, x(t)
    
    # ------------------------------ partial price dynamics ---------------------
    # -------------------------------Technical Indicators ------------------------
    df_features.dropna(inplace=True)
    return df_features

In [4]:
df_features = calc_feature_dataframe(btcusd_6h_historical)
df_features.head()

,close,returns,cum_returns,price_velocity_2,price_velocity_3,price_velocity_4,price_velocity_7,price_velocity_30,price_acceleration_1,price_acceleration_2,...,rolling_mean_velocity_4,rolling_mean_velocity_7,rolling_mean_velocity_30,rolling_mean_acceleration_2,rolling_mean_acceleration_3,rolling_mean_acceleration_4,rolling_mean_acceleration_7,rolling_mean_acceleration_30,log_price,log_returns
time,,,,,,,,,,,,,,,,,,,,,
2013-11-03 00:00:00+00:00,204.65999,0.005651,1.677541,0.003727,0.008674,-0.015907,0.012667,0.124691,-3.954343,-1.190539,...,-0.003926,0.001868,0.006912,-2.671215,-2.181556,-2.306948,-1.153610,0.335800,5.321350,0.005635
2013-11-03 06:00:00+00:00,203.87000,-0.003860,1.671066,0.001769,-0.000147,0.004781,0.001326,0.050876,-1.683094,-0.411605,...,0.001202,0.000256,0.004580,-2.818718,-2.341841,-2.056940,-1.411250,0.244207,5.317483,-0.003867
2013-11-03 12:00:00+00:00,206.75000,0.014127,1.694672,0.010212,0.015921,0.013977,0.008537,0.378333,-4.659742,1.739837,...,0.003501,0.001292,0.012611,-3.171418,-3.432393,-2.921317,-2.066424,0.236572,5.331510,0.014028
2013-11-03 18:00:00+00:00,209.74800,0.014501,1.719246,0.028832,0.024861,0.030652,0.008558,0.090842,0.026472,15.298946,...,0.007605,0.001295,0.003699,-2.316635,-2.105455,-2.567677,-2.220594,0.312215,5.345907,0.014396
2013-11-04 00:00:00+00:00,210.70095,0.004543,1.727057,0.019110,0.033506,0.029517,0.038447,0.095629,-0.686681,0.871288,...,0.007328,0.005425,0.003845,-0.330105,-1.773317,-1.750761,-1.935388,0.322641,5.350440,0.004533


In [17]:
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from sklearn import svm
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split



# Create Class Column with bin logic
bin_labels_5 = ['1', '2', '3', '4', '5']
class_df = df_features.copy()
class_df['return_class'] = pd.cut(df_features.returns,
                              bins=[-np.inf, -.003,-.00125, .00125, .003, np.inf], labels=bin_labels_5)

cleanup_nums = {"return_class": {"1": round(1), "2": round(2), "3": round(3), "4": round(4)}}
class_df.replace(cleanup_nums, inplace=True)

# Create independent variable dataframe
X = class_df.copy()
X.drop(["close", "returns", "return_class"], axis=1, inplace=True)
# Create dependent variable dataframe
y = class_df["return_class"].values.reshape(-1, 1)

# Split into Training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Train Model on training dataset
model = xgb.XGBClassifier(objective='multi:softmax',num_class=5).fit(X_train,y_train)

# Test model on out of sample testing dataset
predictions = model.predict(X_test)
predicted_returns = predictions.reshape(-1,1)
real_returns = y_test.reshape(-1, 1)

# Create Prediction Dataframe
prediction_xgb = pd.DataFrame({
    "Real": real_returns.ravel(),
    "y_test": y_test.ravel(),
    "Predicted": predicted_returns.ravel()
})

# Create accuracy test column to identify errors in prediction
prediction_xgb['Accuracy_Test'] = np.where(prediction_xgb['Real'] == prediction_xgb['Predicted'], True, False)

# Print Value Counts of True and false predictions
print(prediction_xgb['Accuracy_Test'].value_counts())


C:\Users\gdepa\Anaconda3\envs\mantis\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [33]:
X_test

,cum_returns,price_velocity_2,price_velocity_3,price_velocity_4,price_velocity_7,price_velocity_30,price_acceleration_1,price_acceleration_2,price_acceleration_3,price_acceleration_4,...,rolling_mean_velocity_4,rolling_mean_velocity_7,rolling_mean_velocity_30,rolling_mean_acceleration_2,rolling_mean_acceleration_3,rolling_mean_acceleration_4,rolling_mean_acceleration_7,rolling_mean_acceleration_30,log_price,log_returns
time,,,,,,,,,,,,,,,,,,,,,
2015-04-17 06:00:00+00:00,1.834920,-0.041694,-0.018756,-0.018003,0.017084,-0.102469,-0.322017,-2.686535,-1.514097,-1.476264,...,-0.004354,0.002614,-0.003364,-1.185376,9.271478,7.292620,129.341493,62.331987,5.411022,-0.017165
2020-03-10 18:00:00+00:00,64.646721,-0.006375,-0.005159,0.020680,-0.018505,-0.099309,-1.230564,-1.234119,2.561681,-1.177378,...,0.005210,-0.002578,-0.003306,-4.492593,-3.312693,-3.334098,-2.024869,-2.620024,8.972958,0.001904
2014-01-26 00:00:00+00:00,6.663934,-0.008537,0.022545,0.012429,-0.021530,-0.016894,-1.124014,-1.403697,-2.553932,-1.567506,...,0.003233,-0.002923,-0.000392,-1.217220,-2.201060,-1.772223,-2.043491,96.417775,6.700731,0.001206
2015-09-03 18:00:00+00:00,1.860154,-0.007918,0.003399,-0.013481,-0.011160,0.005504,0.186474,0.411949,-1.356060,-6.625064,...,-0.003337,-0.001529,0.000246,-0.565802,-0.936559,-0.620377,-17.879404,-4.988036,5.424680,-0.004314
2014-05-08 06:00:00+00:00,3.623607,0.000439,-0.016212,0.004727,0.023552,-0.014699,-1.946338,-0.897658,-1.400184,-0.660856,...,0.001293,0.003417,-0.000403,-1.770766,-1.774495,-1.306847,-5.006830,0.395674,6.091491,-0.009419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-04-17 06:00:00+00:00,9.683992,0.000380,-0.003142,0.005480,-0.001484,0.003887,-1.904453,-0.925422,-0.051680,-1.501436,...,0.001381,-0.000190,0.000181,-2.040326,-1.829246,-1.793458,-3.990879,-3.670231,7.074495,-0.003753
2015-02-25 00:00:00+00:00,1.948906,0.006973,0.015528,0.015749,-0.033705,0.061458,-1.096662,-0.199911,-2.361293,-1.626078,...,0.003923,-0.004780,0.002724,-0.593650,12.268656,8.937579,4.563796,4.046822,5.471289,-0.000747
2017-03-10 12:00:00+00:00,10.278598,0.054348,0.056263,0.050513,0.096732,-0.000009,4.108633,-15.941267,0.344052,-5.122434,...,0.012581,0.013418,0.000215,3.985893,2.212725,1.354286,-0.427646,-0.648420,7.134085,0.044130


In [23]:
prediction_xgb

,Real,y_test,Predicted,Accuracy_Test
0,1,1,1,True
1,4,4,4,True
2,3,3,3,True
3,1,1,1,True
4,1,1,1,True
...,...,...,...,...
2256,1,1,1,True
2257,3,3,3,True
2258,5,5,5,True
2259,3,3,3,True


In [46]:
X_test.reset_index(inplace=True)
test_df = pd.merge(X_test, prediction_xgb, left_index=True, right_index=True)
test_df.set_index('time', inplace=True)
test_df = pd.merge(test_df, btcusd_6h_historical, how='left', on='time')
test_df = pd.merge(test_df, ethbtc_6h_historical, how='left', on='time')
test_df['btcusd_price'] = test_df['close_x']
test_df['ethbtc_price'] = test_df['close_y']
test_df.drop(columns=['close_x', 'close_y'], inplace=True)
test_df['ethbtc_price'].fillna(method='ffill', inplace=True)
test_df['btcusd_change'] = test_df['btcusd_price'].pct_change()
test_df['ethbtc_change'] = test_df['ethbtc_price'].pct_change()

In [49]:
test_df.tail()

,cum_returns,price_velocity_2,price_velocity_3,price_velocity_4,price_velocity_7,price_velocity_30,price_acceleration_1,price_acceleration_2,price_acceleration_3,price_acceleration_4,...,log_price,log_returns,Real,y_test,Predicted,Accuracy_Test,btcusd_price,ethbtc_price,btcusd_change,ethbtc_change
time,,,,,,,,,,,,,,,,,,,,,
2017-04-17 06:00:00+00:00,9.683992,0.000380,-0.003142,0.005480,-0.001484,0.003887,-1.904453,-0.925422,-0.051680,-1.501436,...,7.074495,-0.003753,1,1,1,True,1181.44700,0.0233,-0.897447,0.0
2015-02-25 00:00:00+00:00,1.948906,0.006973,0.015528,0.015749,-0.033705,0.061458,-1.096662,-0.199911,-2.361293,-1.626078,...,5.471289,-0.000747,3,3,3,True,237.76658,0.0233,-0.798750,0.0
2017-03-10 12:00:00+00:00,10.278598,0.054348,0.056263,0.050513,0.096732,-0.000009,4.108633,-15.941267,0.344052,-5.122434,...,7.134085,0.044130,5,5,5,True,1253.98900,0.0233,4.274034,0.0
2016-02-27 12:00:00+00:00,3.527582,-0.008490,0.013506,0.016210,0.023629,0.022269,-0.958287,-1.340797,1.049484,2.600213,...,6.064634,-0.000340,3,3,3,True,430.36502,0.0233,-0.656803,0.0
2014-11-02 18:00:00+00:00,2.655243,-0.003840,0.012311,-0.003839,-0.059108,-0.181541,252.350057,-3903.621160,-1.153262,-0.936230,...,5.780557,-0.003833,1,1,1,True,323.93959,0.0233,-0.247291,0.0


In [ ]:
test

In [ ]:
def return_analysis(test_df, initial_capital, exchange, rebalance_freq, start, end, fee_perc, tax_fee):
    name_of_model = 'XGB Multi-Classifier'
    initial_capital = float(initial_capital)
    # BTC-USD Trend thresholds
    very_bullish = 5 # if btc price prediction is greater than 
    bullish = 4 # if btc
    nuetral = 3 # if btc is predicting 0 returns
    bearish = 2 # if btc is predicting negative returns
    very_bearish = 1 # if btci predicting very negative returns
    # Set Threshold Weights (create global variables as input...)
    # ***** figure out how to train a model to predict class ranges for trend thresholds
    # ***** figure out how to train a model to predict best allocation wts given the class prediction
    btc_wts = [.2,.4,.5,.5,.4]
    usd_wts = [.8,.6,.4,.2, 0]
    alt_wts = [0,0,.1,.3,.6]
    # Create balance lists to populate when looping through dataframe
    bucket_1_position = []
    bucket_2_position = []
    bucket_3_position = []
    usd_balance = []
    btc_balance = []
    time = []
    for index, row in test_df.iterrows():
        # Strategy Logic    
        if row['predicted_class'] == 5: # Very Bull
            bucket_1_position.append(usd_wts[4])
            bucket_2_position.append(btc_wts[4])
            bucket_3_position.append(alt_wts[4])
            balance_usd = row['']* usd_wts[4]
            balance_btc = row['']
            
            time.append(row['time'])
        elif row['predicted_class'] == 4: # Bull
            bucket_1_position.append(usd_wts[3])
            bucket_2_position.append(btc_wts[3])
            bucket_3_position.append(alt_wts[3])
            time.append(row['time'])          
        elif row['predicted_class'] == 3: # Neutral
            bucket_1_position.append(usd_wts[2])
            bucket_2_position.append(btc_wts[2])
            bucket_3_position.append(alt_wts[2])
            time.append(row['time'])
        elif row['predicted_class'] == 2: # Bear
            bucket_1_position.append(usd_wts[1])
            bucket_2_position.append(btc_wts[1])
            bucket_3_position.append(alt_wts[1])
            time.append(row['time'])
        elif row['predicted_class'] == 1: # Very Bear
            bucket_1_position.append(usd_wts[0])
            bucket_2_position.append(btc_wts[0])
            bucket_3_position.append(alt_wts[0])
            time.append(row['time'])

    # Create positions dataframe  
    positions_df = pd.DataFrame(list(zip(time, bucket_1_position, bucket_2_position, bucket_3_position)),
                                 columns =['time', 'bucket_1_position', 'bucket_2_position', 'bucket_3_position'])   
    
    # Merge positions dataframe with backtest dataframe
    final_backtest_df = pd.merge(test_df, positions_df, how='left', on='time')
    # Market Dynamics Columns
    final_backtest_df['btc_price_change'] = backtest_df['btc_price'].pct_change()
    final_backtest_df['eth_price_change'] = backtest_df['ethbtc_price'].pct_change()
    # Portfolio Columns
    final_backtest_df['portfolio_balance_usd'] = final_backtest_df['bucket_1_position'] 
    final_backtest_df['portfolio_returns_usd'] =  final
    final_backtest_df['portfolio_cum_returns_usd'] = 
    final_backtest_df['portfolio_balance_btc'] = 
    final_backtest_df['portfolio_returns_btc'] =
    final_backtest_df['portfolio_cum_returns_btc'] =
    # Position Columns
    final_backtest_df['bucket_1_position_change'] = backtest_df['bucket_1_position'].diff()
    final_backtest_df['bucket_2_position_change'] = backtest_df['bucket_2_position'].diff()
    final_backtest_df['bucket_3_position_change'] = backtest_df['bucket_3_position'].diff()
    
    
    
    # ---------------------------------- Calc Portfolio Performance ------------------------
    
    #backtest_results_df1['alpha'] = backtest_results_df1['usd_cum_returns'] - backtest_results_df1['btcusd_cum_returns']
    #backtest_results_df1['usd_rolling_returns_wk'] = backtest_results_df1['usd_perc_change'].rolling(7).sum()
    #backtest_results_df1['btc_rolling_returns_wk'] = backtest_results_df1['btc_perc_change'].rolling(7).sum()
    #backtest_results_df1['usd_rolling_vol_wk'] = backtest_results_df1['usd_perc_change'].rolling(7).std()
    #backtest_results_df1['btc_rolling_vol_wk'] = backtest_results_df1['btc_perc_change'].rolling(7).std()
    #backtest_results_df1['usd_rolling_returns_mo'] = backtest_results_df1['usd_perc_change'].rolling(7).sum()
    #backtest_results_df1['btc_rolling_returns_mo'] = backtest_results_df1['btc_perc_change'].rolling(7).sum()
    #backtest_results_df1['usd_rolling_vol_mo'] = backtest_results_df1['usd_perc_change'].rolling(30).std()
    #backtest_results_df1['btc_rolling_vol_mo'] = backtest_results_df1['btc_perc_change'].rolling(30).std()
    #
    #backtest_results_df1['usd_rolling_sharpe_wk'] = backtest_results_df1['usd_rolling_returns_wk']/backtest_results_df1['usd_rolling_vol_wk']
    #backtest_results_df1['btc_rolling_sharpe_wk'] = backtest_results_df1['btc_rolling_returns_wk']/backtest_results_df1['btc_rolling_vol_wk']
    #backtest_results_df1['usd_rolling_sharpe_mo'] = backtest_results_df1['usd_rolling_returns_mo']/backtest_results_df1['usd_rolling_vol_mo']
    #backtest_results_df1['btc_rolling_sharpe_mo'] = backtest_results_df1['btc_rolling_returns_mo']/backtest_results_df1['btc_rolling_vol_mo']
    
    # More Risk Measures
        # Max Drawdown 
        # Skewness
        # Kurtosis
        # VAR
        # CVAR
        # Time under water   
    usd_metrics= [
        'initial_capital',
        'cumulative_returns',
        'annualized_returns',
        'mthly_returns',
        'annual_vol',
        'mthly_vol',
        'annual_sharpe',
        'mthly_sharpe',
        'annual_sortino',
        'mthly_sortino',
    ]
    
    btc_metrics = [
        
        'initial_capital',
        'cumulative_returns',
        'annualized_returns',
        'mthly_returns',
        'annual_vol',
        'mthly_vol',
        'annual_sharpe',
        'mthly_sharpe',
        'annual_sortino',
        'mthly_sortino',
    ]
    
    columns = ['Backtest']
    
    port_eval_df_usd = pd.DataFrame(index=usd_metrics, columns=columns)
    
    port_eval_df_usd.loc['initial_capital'] = initial_capital
    port_eval_df_usd.loc['cumulative_returns'] = final_backtest_df['portfolio_cum_returns_usd'].iloc[-1]
    port_eval_df_usd.loc['annualized_returns'] = final_backtest_df['portfolio_returns_usd'].mean() *365  #Crypto trades 365 days per year
    port_eval_df_usd.loc['mthly_returns'] = final_backtest_df['portfolio_returns_usd'].mean() *30
    port_eval_df_usd.loc['annual_vol'] = final_backtest_df['portfolio_returns_usd'].std() * np.sqrt(365)  #Crypto trades 365 days per year
    port_eval_df_usd.loc['mthly_vol'] = final_backtest_df['portfolio_returns_usd'].std() * np.sqrt(30)
    port_eval_df_usd.loc['annual_sharpe'] = port_eval_df_usd.loc['annualized_returns']/port_eval_df_usd.loc['annual_vol']
    port_eval_df_usd.loc['mthly_sharpe'] = port_eval_df_usd.loc['mthly_returns']/port_eval_df_usd.loc['mthly_vol']
        
    
    port_eval_df_btc = pd.DataFrame(index=btc_metrics, columns=columns)
    port_eval_df_btc.loc['initial_capital'] = final_backtest_df['portfolio_balance_btc'].iloc[0]
    port_eval_df_btc.loc['cumulative_returns'] = final_backtest_df['portfolio_cum_returns_btc'].iloc[-1]
    port_eval_df_btc.loc['annualized_returns'] = final_backtest_df['portfolio_returns_btc'].mean() *365  #Crypto trades 365 days per year
    port_eval_df_btc.loc['mthly_returns'] = final_backtest_df['portfolio_returns_btc'].mean() *30
    port_eval_df_btc.loc['annual_vol'] = final_backtest_df['portfolio_returns_btc'].mean() * np.sqrt(365) #Crypto trades 365 days per year
    port_eval_df_btc.loc['mthly_vol'] = final_backtest_df['portfolio_returns_btc'].mean() * np.sqrt(30)
    port_eval_df_btc.loc['annual_sharpe'] = port_eval_df_btc.loc['annualized_returns']/port_eval_df_btc.loc['annual_vol']
    port_eval_df_btc.loc['mthly_sharpe'] = port_eval_df_btc.loc['mthly_returns']/port_eval_df_btc.loc['mthly_vol']
    
    # Calculate Sortino Ratios
    sortino_ratio_df = final_backtest_df[['portfolio_returns_usd','portfolio_returns_btc']].copy()
    sortino_ratio_df.loc[:, "downside_usd"] = 0
    sortino_ratio_df.loc[:,"downside_btc"] = 0
    target = 0
    mask_usd = sortino_ratio_df['portfolio_returns_usd'] < target
    mask_btc = sortino_ratio_df['portfolio_returns_btc'] < target
    
    sortino_ratio_df.loc[mask_usd, "portfolio_returns_usd"] = (sortino_ratio_df["portfolio_returns_usd"]**2)
    sortino_ratio_df.loc[mask_btc, "portfolio_returns_btc"] = (sortino_ratio_df["portfolio_returns_btc"]**2)
    
    down_stdev_usd_annual = np.sqrt(sortino_ratio_df['downside_usd'].mean()) * np.sqrt(365)
    down_stdev_btc_annual = np.sqrt(sortino_ratio_df['downside_btc'].mean()) * np.sqrt(365)
    down_stdev_btc_mthly = np.sqrt(sortino_ratio_df['downside_btc'].mean()) * np.sqrt(30)
    down_stdev_usd_mthly = np.sqrt(sortino_ratio_df['downside_usd'].mean()) * np.sqrt(30)
    
    expected_return_usd_annual = sortino_ratio_df['portfolio_returns_usd'].mean()*365
    expected_return_btc_annual = sortino_ratio_df['portfolio_returns_btc'].mean()*365
    sortino_ratio_usd_annual = expected_return_usd_annual/down_stdev_usd_annual
    sortino_ratio_btc_annual = expected_return_btc_annual/down_stdev_btc_annual
    
    expected_return_usd_mthly = sortino_ratio_df['portfolio_returns_usd'].mean()*30
    expected_return_btc_mthly = sortino_ratio_df['portfolio_returns_btc'].mean()*30
    
    sortino_ratio_usd_mthly = expected_return_usd_mthly/down_stdev_usd_mthly
    sortino_ratio_btc_mthly = expected_return_btc_mthly/down_stdev_btc_mthly
    
    port_eval_df_usd.loc['annual_sortino'] = sortino_ratio_usd_annual
    port_eval_df_usd.loc['mthly_sortino'] = sortino_ratio_usd_mthly
    port_eval_df_btc.loc['annual_sortino'] = sortino_ratio_btc_annual
    port_eval_df_btc.loc['mthly_sortino'] = sortino_ratio_usd_mthly
    
    
    
    
    # --------------------------------------------- print Performance stats ------------------------------------------
    # Print out Performance Tables
    print(port_eval_df_usd)
    print(port_eval_df_btc)
    
    # Risk Visualizations    
    #final_backtest_df['rolling_sharpe'] !!!! NEED TO MAKE SURE THERE IS A ROW EVERY 6 HOURS
    final_backtest_df[['portfolio_returns_usd', 'portfolio_returns_btc']]
    # Reward Measures
    final_backtest_df['portfolio_cum_returns_usd'].plot()
    final_backtest_df['portfolio_cum_returns_btc'].plot()  
    final_backtest_df['portfolio_returns_usd'].plot()
    final_backtest_df['portfolio_returns_btc'].plot()
    


        
    
    